## 펑션 콜링 기초

현재시간을 구하는 것을 해보면서 펑션 콜링에 대해서 알아보자

In [1]:
from time_functions import get_current_time, tools 
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

In [2]:
load_dotenv(dotenv_path="../.env")
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

In [3]:
def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",  # 응답 생성에 사용할 모델 지정
        messages=messages,  # 대화 기록을 입력으로 전달
        tools=tools,  # 사용 가능한 도구 목록 전달
    )
    return response  # 생성된 응답 내용 반환

In [4]:
messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},
]

user_input = "서울은 지금 몇 시지?"

messages.append({"role": "user", "content": user_input})

ai_response = get_ai_response(messages, tools=None)
ai_message = ai_response.choices[0].message
print(ai_message)

print("AI\t: " + ai_message.content)

ChatCompletionMessage(content='죄송하지만, 현재 시각을 실시간으로 확인할 수 없습니다. 서울의 현재 시각을 확인하려면 인터넷 검색이나 스마트폰, 컴퓨터 등의 시계를 참고해 주세요.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
AI	: 죄송하지만, 현재 시각을 실시간으로 확인할 수 없습니다. 서울의 현재 시각을 확인하려면 인터넷 검색이나 스마트폰, 컴퓨터 등의 시계를 참고해 주세요.


### 펑션 콜링 해보기

In [5]:
from time_functions import get_current_time, tools

messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},
]

user_input = "서울은 지금 몇 시지?"

messages.append({"role": "user", "content": user_input})

ai_response = get_ai_response(messages, tools=tools)
ai_message = ai_response.choices[0].message
print(ai_message)

print("AI\t: " + ai_message.content)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_J6LxYddMevtzzE6uYfmRUms9', function=Function(arguments='{"timezone":"Asia/Seoul"}', name='get_current_time'), type='function')])


TypeError: can only concatenate str (not "NoneType") to str

In [10]:
from time_functions import get_current_time, tools

messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},
]

user_input = "서울은 지금 몇 시지?"

messages.append({"role": "user", "content": user_input})

ai_response = get_ai_response(messages, tools=tools)
ai_message = ai_response.choices[0].message
print(ai_message) # GPT 응답 확인용

tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls를 가져옵니다.
if tool_calls:  # tool_calls가 있는 경우
    for tool_call in tool_calls:
        tool_name = tool_call.function.name # 실행해야한다고 판단한 함수명 받기
        tool_call_id = tool_call.id         # tool_call 아이디 받기    
        arguments = json.loads(tool_call.function.arguments) # (1) 문자열을 딕셔너리로 변환    
        
        if tool_name == "get_current_time":  # 만약 tool_name이 "get_current_time"이라면
            messages.append({
                "role": "function",  # role을 "function"으로 설정
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": get_current_time(timezone=arguments['timezone']),  # 타임존 추가
            })
    messages.append({"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."})  # 함수 실행 완료 메시지 추가
    ai_response = get_ai_response(messages, tools=tools) # 다시 GPT 응답 받기
    ai_message = ai_response.choices[0].message

messages.append(ai_message)  # AI 응답을 대화 기록에 추가하기

print("AI\t: " + ai_message.content)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_W79eK70YPfQX7FQZ2i1FYgFN', function=Function(arguments='{"timezone":"Asia/Seoul"}', name='get_current_time'), type='function')])
2025-08-14 22:33:27 Asia/Seoul
AI	: 현재 서울의 시간은 2025년 8월 14일 오후 10시 33분입니다.
